# Dogs vs. Cats

https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

In [ ]:
import numpy as np
import keras
from keras.engine import topology
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
import h5py

In [ ]:
IMG_SIZE = (224, 224)  # размер входного изображения сети

## Функции загрузки данных

In [ ]:
import re
from random import shuffle
from glob import glob

train_files = glob('../input/dogs-vs-cats-redux-kernels-edition/train/*.jpg')
test_files = glob('../input/dogs-vs-cats-redux-kernels-edition/test/*.jpg')

# загружаем входное изображение и предобрабатываем
def load_image(path, target_size=IMG_SIZE):
    img = load_img(path, target_size=target_size)  # загрузка и масштабирование изображения
    array = img_to_array(img)
    return preprocess_input(array)  # предобработка для VGG16

# генератор для последовательного чтения обучающих данных с диска
def fit_generator(files, batch_size=32):
    while True:
        shuffle(files)
        for k in range(len(files) // batch_size):
            i = k * batch_size
            j = i + batch_size
            if j > len(files):
                j = - j % len(files)
            x = np.array([load_image(path) for path in files[i:j]])
            y = np.array([1. if re.match('.*/dog\.\d', path) else 0. for path in files[i:j]])
            yield (x, y)

# генератор последовательного чтения тестовых данных с диска
def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])

## Визуализируем примеры для обучения

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(20, 20))
for i, path in enumerate(train_files[:3], 1):
    subplot = fig.add_subplot(i // 5 + 1, 5, i)
    plt.imshow(plt.imread(path));
    subplot.set_title('%s' % path.split('/')[-1]);

## Загружаем предобученную модель

In [ ]:
#base_model -  объект класса keras.models.Model (Functional Model)
base_model = VGG16(include_top = False,
                   weights = None,
                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
def load_split_weights(model, model_path_pattern):  
    model_f = h5py.File(model_path_pattern, "r")
    topology.load_weights_from_hdf5_group_by_name(model_f, model.layers)
    
    return model

In [ ]:
keras_models_dir = '../input/full-keras-pretrained-no-top'
model_path_pattern = keras_models_dir + "/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5" 
base_model = load_split_weights(base_model, model_path_pattern = model_path_pattern)

In [ ]:
# base_model = MobileNetV2(include_top = False,
#                   weights = 'imagenet',
#                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3))

In [ ]:
# фиксируем все веса предобученной сети, кроме последнего
for layer in base_model.layers[:-1]:
    layer.trainable = False

In [ ]:
base_model.summary()

## Добавляем полносвязный слой

In [ ]:
x = base_model.layers[-1].output
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1,  # один выход
                activation='sigmoid',  # функция активации  
                kernel_regularizer=keras.regularizers.l1(1e-4))(x)
model = Model(inputs=base_model.input, outputs=x)

## Выводим архитектуру модели

In [ ]:
model.summary()

## Компилируем модель и запускаем обучение

In [ ]:
#opt = keras.optimizers.Adadelta(lr=1.0,rho=0.9)
opt = keras.optimizers.Adagrad(lr=0.001, epsilon=None, decay =0.0)
model.compile(optimizer=opt, #'adam', 
              loss='binary_crossentropy',  # функция потерь binary_crossentropy (log loss
              metrics=['accuracy'])

In [ ]:
shuffle(train_files)  # перемешиваем обучающую выборку

train_val_split = 500  # число изображений в валидационной выборке

validation_data = next(fit_generator(train_files[:train_val_split], train_val_split))

# запускаем процесс обучения
model.fit_generator(fit_generator(train_files[train_val_split:]),  # данные читаем функцией-генератором
        steps_per_epoch=10,  # число вызовов генератора за эпоху
        epochs=150,  # число эпох обучения
        validation_data=validation_data)

In [ ]:
model.save('my_vgg19_cats_and_dogs_1.h5')

## Предсказания на проверочной выборке

In [ ]:
pred = model.predict_generator(predict_generator(test_files), len(test_files), max_queue_size=500)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(20, 20))
for i, (path, score) in enumerate(zip(test_files[80:][:10], pred[80:][:10]), 1):
    subplot = fig.add_subplot(i // 5 + 1, 5, i)
    plt.imshow(plt.imread(path));
    subplot.set_title('%.3f' % score);

## Готовим данные для сабмита

In [ ]:
with open('submit_cats_dogs_vgg16_1.txt', 'w') as dst:
    dst.write('id,label\n')
    for path, score in zip(test_files, pred):
        dst.write('%s,%f\n' % (re.search('(\d+)', path).group(0), score))